In [1]:
dataset = []
with open('cat-facts.txt', 'r') as file:
    dataset = file.readlines()
    print(f'Loaded {len(dataset)} entries')

Loaded 150 entries


In [2]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

VECTOR_DB = []

def add_chunk_to_database(chunk):
    embedding= ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
    VECTOR_DB.append((chunk, embedding))

In [3]:
for i, chunk in enumerate(dataset):
    add_chunk_to_database(chunk=chunk)

In [11]:
print(len(VECTOR_DB))

150


In [12]:
def cosine_similarity(a, b):
    dot_product = sum([x * y for x, y in zip(a,b)])
    norm_a = sum([x ** 2 for x in a]) ** 0.5
    norm_b = sum([x ** 2 for x in b]) ** 0.5
    return dot_product / (norm_a * norm_b)

In [13]:
def retrieve(query, top_n=3):
    query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
    similarities = []
    for chunk, embedding in VECTOR_DB:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))

    similarities.sort(key=lambda x: x[1], reverse=True)

    return similarities[:top_n]

In [14]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge: ')
for chunk, similarity in retrieved_knowledge:
    print(f' - (similarity: {similarity:.2f}) {chunk}')

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieced of context to answer the question. Don't ake up any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
'''

Retrieved knowledge: 
 - (similarity: 0.78) A cat can travel at a top speed of approximately 31 mph (49 km) over a short distance.

 - (similarity: 0.67) Cats are extremely sensitive to vibrations. Cats are said to detect earthquake tremors 10 or 15 minutes before humans can.

 - (similarity: 0.67) Researchers are unsure exactly how a cat purrs. Most veterinarians believe that a cat purrs by vibrating vocal folds deep in the throat. To do this, a muscle in the larynx opens and closes the air passage about 25 times per second.



In [15]:
stream = ollama.chat(
    model=LANGUAGE_MODEL,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query}
    ],
    stream=True
)

print('Chatbot response: ')
for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)

Chatbot response: 
According to the information you provided:

* A cat can travel at approximately 31 mph (49 km) over a short distance.

In [4]:
import chromadb

chroma_client = chromadb.HttpClient(host='localhost', port=8000)

In [13]:
collection = chroma_client.create_collection(name="test")
print(chroma_client.list_collections())

['test']


In [14]:
ids = [str(i) for i, _ in enumerate(dataset)]
collection.add(documents=dataset, ids=ids)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:10<00:00, 8.14MiB/s]


In [ ]:
results = collection.query(
    query_texts=[input_query],
    n_results=3
)

print(results)

{'ids': [['20', '78', '108']], 'distances': [[0.5415200591087341, 0.8134765028953552, 0.9529806028626654]], 'embeddings': None, 'metadatas': [[None, None, None]], 'documents': [['A cat can travel at a top speed of approximately 31 mph (49 km) over a short distance.\n', 'A cat’s heart beats nearly twice as fast as a human heart, at 110 to 140 beats a minute.\n', 'A cat can jump even seven times as high as it is tall.\n']], 'uris': None, 'data': None, 'included': ['distances', 'documents', 'metadatas']}


In [18]:
distances = results['distances'][0]
documents = results['documents'][0]

for i in range(3):
    print(f' - (distance: {distances[i]:.2f}) {documents[i]}')

 - (distance: 0.54) A cat can travel at a top speed of approximately 31 mph (49 km) over a short distance.

 - (distance: 0.81) A cat’s heart beats nearly twice as fast as a human heart, at 110 to 140 beats a minute.

 - (distance: 0.95) A cat can jump even seven times as high as it is tall.



In [37]:
from chromadb.utils.embedding_functions.ollama_embedding_function import (
    OllamaEmbeddingFunction,
)

ollama_ef = OllamaEmbeddingFunction(
    url="http://localhost:11434",
    model_name="llama2",
)

embeddings = ollama_ef(["This is my first text to embed",
                        "This is my second document"])

JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [38]:
from typing import cast
from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        # embed the documents somehow
        embeddings = ollama.embed(model=EMBEDDING_MODEL, input=input)['embeddings'][0]
        return cast(Embeddings, embeddings)

In [41]:
ef = MyEmbeddingFunction()

collection2 = chroma_client.create_collection(name="test2", embedding_function=ef)

In [43]:
ids = [str(i) for i, _ in enumerate(dataset)]
collection2.add(documents=dataset, ids=ids)

Exception: {"error":"IndexError('list index out of range')"} (trace ID: 0)

In [6]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
import chromadb.utils.embedding_functions as embedding_functions


openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small"
)

In [9]:
collection2 = chroma_client.create_collection(name='test2', embedding_function=openai_ef)

In [10]:
ids = [str(i) for i, _ in enumerate(dataset)]
collection2.add(documents=dataset, ids=ids)

In [16]:
results = collection2.query(
    query_texts=[input_query],
    n_results=3
)

distances = results['distances'][0]
documents = results['documents'][0]

for i in range(3):
    print(f' - (distance: {distances[i]:.2f}) {documents[i]}')

 - (distance: 0.66) A cat can travel at a top speed of approximately 31 mph (49 km) over a short distance.

 - (distance: 0.91) A cat’s heart beats nearly twice as fast as a human heart, at 110 to 140 beats a minute.

 - (distance: 0.98) A cat can jump up to five times its own height in a single bound.

